<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/Apache_Solr_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and run Solr on a Google Colab instance


In [1]:
!java -version

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [2]:
!wget -nc https://downloads.apache.org/lucene/solr/8.10.1/solr-8.10.1.tgz
!tar xf solr-8.10.1.tgz

--2021-10-25 21:07:51--  https://downloads.apache.org/lucene/solr/8.10.1/solr-8.10.1.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217254401 (207M) [application/x-gzip]
Saving to: ‘solr-8.10.1.tgz’

solr-8.10.1.tgz     100%[===================>] 207.19M  25.9MB/s    in 8.9s    

2021-10-25 21:08:00 (23.3 MB/s) - ‘solr-8.10.1.tgz’ saved [217254401/217254401]



In [31]:
!(cd solr-8.10.1; ./bin/solr -e cloud -noprompt -force)


Welcome to the SolrCloud example!

Starting up 2 Solr nodes for your example SolrCloud cluster.

Solr home directory /content/solr-8.10.1/example/cloud/node1/solr already exists.
/content/solr-8.10.1/example/cloud/node2 already exists.

Starting up Solr on port 8983 using command:
"bin/solr" start -cloud -p 8983 -s "example/cloud/node1/solr" -force

Waiting up to 180 seconds to see Solr running on port 8983 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]  
Started Solr server on port 8983 (pid=985). Happy searching!

    
Starting up Solr on port 7574 using command:
"bin/solr" start -cloud -p 7574 -s "example/cloud/node2/solr" -z localhost:9983 -force

Waiting up to 180 seconds to see Solr running on port 7574 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [

## Use solr

In [8]:
%pip install -q pysolr kazoo

In [7]:
import requests
import pysolr
import json
solr_zk_hosts = ['localhost:9983']
zookeeper     = pysolr.ZooKeeper(",".join(solr_zk_hosts))
collection    = list(zookeeper.collections.keys())[0]
solr          = pysolr.SolrCloud(zookeeper,collection,timeout=60)


In [42]:
resp = requests.get("http://localhost:8983/solr/admin/collections?action=DELETE&name=collection1").json()
resp = requests.get("http://localhost:8983/solr/admin/collections?action=CREATE&name=collection1&numShards=1").json()
resp = requests.post("http://localhost:8983/solr/collection1/schema", 
                  json={
                      "add-field": [
                         {"name":"timestamp", "type":"pdate", "indexed":"True","stored":"True","default":"NOW"},
                        ]
                      }
                  ).json()
print(json.dumps(resp,indent=1))

{
 "responseHeader": {
  "status": 0,
  "QTime": 425
 }
}


In [40]:
resp = requests.get("http://localhost:8983/solr/collection1/schema")
print(json.dumps(resp.json()['schema']['fields'],indent=1))

[
 {
  "name": "_nest_path_",
  "type": "_nest_path_"
 },
 {
  "name": "_root_",
  "type": "string",
  "docValues": false,
  "indexed": true,
  "stored": false
 },
 {
  "name": "_text_",
  "type": "text_general",
  "multiValued": true,
  "indexed": true,
  "stored": false
 },
 {
  "name": "_version_",
  "type": "plong",
  "indexed": false,
  "stored": false
 },
 {
  "name": "id",
  "type": "string",
  "multiValued": false,
  "indexed": true,
  "required": true,
  "stored": true
 },
 {
  "name": "timestamp",
  "type": "pdate",
  "indexed": true,
  "default": "NOW"
 }
]


In [34]:
resp.json()

{'error': {'code': 400,
  'details': [{'add-field': {'default': 'NOW',
     'indexed': 'True',
     'name': 'timestamp',
     'type': 'pdate'},
    'errorMessages': ["Field 'timestamp' already exists.\n"]}],
  'metadata': ['error-class',
   'org.apache.solr.api.ApiBag$ExceptionWithErrObject',
   'root-error-class',
   'org.apache.solr.api.ApiBag$ExceptionWithErrObject'],
  'msg': "error processing commands, errors: [{add-field={name=timestamp, type=pdate, indexed=True, default=NOW}, errorMessages=[Field 'timestamp' already exists.\n]}], "},
 'responseHeader': {'QTime': 82, 'status': 400}}

In [48]:
!nmap -p 9000-9999 localhost


Starting Nmap 7.60 ( https://nmap.org ) at 2021-10-25 21:39 UTC
Nmap scan report for localhost (127.0.0.1)
Host is up (0.0000050s latency).
Other addresses for localhost (not scanned): ::1
Not shown: 999 closed ports
PORT     STATE SERVICE
9983/tcp open  unknown

Nmap done: 1 IP address (1 host up) scanned in 1.62 seconds


Expose solr using ngrok

In [ ]:
!wget -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

# !(sh -c "nohup ./ngrok http 8983 &") ; echo "started ngrok"1
# !curl http://localhost:4040/api/tunnels

File ‘ngrok-stable-linux-amd64.zip’ already there; not retrieving.

Archive:  ngrok-stable-linux-amd64.zip


In [ ]:
%pip install pysolr

In [ ]:
import pysolr

solr = pysolr.Solr('http://localhost:8983/solr/test_collection')

In [ ]:
solr.add([{'id':1,'text':'hello'}])


'{\n  "responseHeader":{\n    "rf":1,\n    "status":0,\n    "QTime":459}}\n'

In [ ]:
results = solr.search('hello')
results.raw_response

{'response': {'docs': [], 'numFound': 0, 'numFoundExact': True, 'start': 0},
 'responseHeader': {'QTime': 1,
  'params': {'q': 'hello', 'wt': 'json'},
  'status': 0,
  'zkConnected': True}}

In [26]:
!curl http://localhost:8983/solr/admin/collections?action=OVERSEERSTATUS

{
  "responseHeader":{
    "status":0,
    "QTime":31},
  "leader":"172.28.0.2:8983_solr",
  "overseer_queue_size":0,
  "overseer_work_queue_size":0,
  "overseer_collection_queue_size":2,
  "overseer_operations":[
    "leader",{
      "requests":2,
      "errors":0,
      "avgRequestsPerSecond":0.0046353395315971305,
      "5minRateRequestsPerSecond":0.04982600162237955,
      "15minRateRequestsPerSecond":0.12553928476507112,
      "avgTimePerRequest":0.1775740744746507,
      "medianRequestTime":0.157514,
      "75thPcRequestTime":0.157514,
      "95thPcRequestTime":0.376659,
      "99thPcRequestTime":0.376659,
      "999thPcRequestTime":0.376659},
    "delete",{
      "requests":1,
      "errors":0,
      "avgRequestsPerSecond":0.003556563443032423,
      "5minRateRequestsPerSecond":0.0799699308689695,
      "15minRateRequestsPerSecond":0.14734279502767977,
      "avgTimePerRequest":0.089957,
      "medianRequestTime":0.089957,
      "75thPcRequestTime":0.089957,
      "95thPcRequest